#### 1.Load the dataset and explore the variables.

#### 2.We will try to predict variable Churn using a logistic regression on variables tenure, SeniorCitizen,MonthlyCharges.

#### 3.Extract the target variable.